# Data extraction of trips using Dask dataframe

# Purpose
As a first step the time series data will be divided into trips, as a data reduction. Energy consumption can be calculated for each trip together with other aggregated quantities such as mean values, standard deviations etc. This will be used to analyze how much trips differ from each other over the year.

But the file is larger than the memory can take so this solution uses a Dask dataframe instead.

# Methodology
* Loop over the dask dataframe partitions and number the trips, save to partquet in each loop.

# Setup

In [1]:
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe

sys.path.append('../')
from src.visualization import visualize
import scipy.integrate

try:
    import trip_statistics
except:
    sys.path.append('../../../../src/models/pipelines/longterm/scripts/prepdata/trip_statistics')
    import trip_statistics

try:
    import trip_id,prepare_dataset,trips
except:
    sys.path.append('../../../../src/models/pipelines/longterm/scripts/prepdata/trip')
    import trip_id,prepare_dataset,trips

try:
    import clean_statistics
except:
    sys.path.append('../../../../src/models/pipelines/longterm/scripts/prepdata/clean_statistics')
    import clean_statistics

Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 463 ('figure.dpi       : 100        ## figure dots per inch')


## Parameters

In [2]:
dataset_name='id'

In [3]:
from dask.distributed import Client, progress, TimeoutError
client = Client(n_workers=8, threads_per_worker=1, memory_limit='4GB')
client

Client Scheduler: tcp://127.0.0.1:49239 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 29.80 GiB


In [4]:
file_path = f'{dataset_name}.parquet'
df = dask.dataframe.read_parquet(file_path)

In [5]:
df.head()

,latitude,longitude,heading,P1,P2,P3,P4,P,cog,sin_pm1,sin_pm2,sin_pm3,sin_pm4,cos_pm1,cos_pm2,cos_pm3,cos_pm4,power_heeling,sog,trip_no,trip_time,reversing,trip_direction,time
0,56.0331,12.61723,77.7,146.0,123.0,148.0,164.0,581.0,89.04,-0.20230,-0.15491,0.01044,-0.01880,-0.98050,-0.99164,-1.0,-0.99954,0.0,0.216067,0.0,0.0,False,0,2020-01-01 08:31:19+00:00
1,56.0331,12.61723,77.6,164.0,166.0,150.0,162.0,642.0,86.49,-0.10016,-0.09088,0.02536,-0.00851,-0.98849,-0.99707,-1.0,-0.99966,0.0,0.205778,0.0,1.0,False,0,2020-01-01 08:31:20+00:00
2,56.0331,12.61723,77.5,171.0,177.0,146.0,162.0,656.0,84.19,-0.07849,-0.02191,0.02719,-0.00839,-0.99469,-0.99997,-1.0,-0.99969,0.0,0.246933,0.0,2.0,False,0,2020-01-01 08:31:21+00:00
3,56.0331,12.61724,77.4,182.0,186.0,152.0,159.0,679.0,81.75,-0.05557,0.00128,0.02710,-0.00848,-0.99591,-0.99997,-1.0,-0.99969,0.0,0.288089,0.0,3.0,False,0,2020-01-01 08:31:22+00:00
4,56.0331,12.61724,77.3,203.0,205.0,150.0,158.0,716.0,80.01,-0.04800,0.00131,0.02771,-0.00830,-0.99615,-0.99994,-1.0,-0.99954,0.0,0.288089,0.0,4.0,False,0,2020-01-01 08:31:23+00:00


In [6]:
path = f'{dataset_name}_statistics.parquet'
df_statistics = trip_statistics.process(df=df, path=path)

In [7]:
df_statistics.head()

,latitude,longitude,heading,P1,P2,P3,P4,P,cog,sin_pm1,sin_pm2,sin_pm3,sin_pm4,cos_pm1,cos_pm2,cos_pm3,cos_pm4,power_heeling,sog,trip_no,trip_time,reversing,trip_direction,start_time,end_time,start_index,end_index
trip_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,56.037954,12.657033,66.737998,414.949513,445.467671,360.205492,355.449956,1576.072631,72.063481,-0.018140,0.014034,-0.211319,-0.166250,-0.672925,-0.675743,-0.809043,-0.803045,3.197520,4.410812,0.0,564.010629,0.0,0,2020-01-01 08:31:19+00:00,2020-01-01 08:50:15+00:00,0,1128
3.0,56.035049,12.653717,260.582487,493.094571,522.021016,496.622592,468.592820,1980.330998,255.338827,0.021302,0.097969,0.098880,0.087748,0.720291,0.731599,0.716466,0.718973,4.789842,4.354430,3.0,570.503503,1.0,1,2020-01-01 10:01:22+00:00,2020-01-01 10:20:25+00:00,3454,4595
5.0,56.035371,12.654587,260.613497,472.056091,507.476775,546.602103,512.732691,2038.867660,254.137897,-0.115793,-0.048235,-0.026392,-0.035503,0.758520,0.765465,0.733363,0.739276,0.000000,4.378689,5.0,570.003506,1.0,1,2020-01-01 11:02:59+00:00,2020-01-01 11:22:01+00:00,5742,6882
10.0,56.036796,12.657713,66.265978,398.036804,423.712747,369.557451,366.584381,1557.891382,72.932343,-0.017451,0.014261,-0.193787,-0.144535,-0.674397,-0.675625,-0.812512,-0.809148,3.508079,4.384069,10.0,556.500000,0.0,0,2020-01-01 13:32:11+00:00,2020-01-01 13:50:44+00:00,11094,12207
15.0,56.034838,12.654486,259.095478,402.555652,429.669565,503.796522,477.806087,1813.827826,254.506217,-0.009575,0.077045,0.076041,0.062345,0.751532,0.760920,0.733007,0.734754,0.000000,4.339030,15.0,574.500000,1.0,1,2020-01-01 16:03:51+00:00,2020-01-01 16:23:00+00:00,16730,17879


In [8]:
client.close()